In [2]:
# Para no tener que reiniciar el kernel
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_api as sa
import numpy as np
import pandas as pd
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select

# Para trabajar con Mongo
# ------------------------------------------------------------------------------
import pymongo
import re

In [3]:
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']
actores = imdb['Actores']

In [24]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/")

for actor in tqdm(list(actores.find())):
    if  type(actor['conocido']) is float:
        continue
    elif len(actor['conocido'])<1:
        continue

    elif type(actor['conocido'][0]) is float:
        continue
    elif len(actor['conocido'][0]) == 9:
        continue


    try:
        busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor['nombre'], Keys.ENTER)
        driver.implicitly_wait(3)
        driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-da7f4a08-0.eryxLJ > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()

    except:
        driver.close()
        #Abrimos el Chrome
        driver = webdriver.Chrome()

        # Maximizar la ventana del navegador
        driver.maximize_window()

        # Navegar a la página web de IMDB 
        driver.get("https://www.imdb.com")
        busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor['nombre'], Keys.ENTER)
        try:
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-da7f4a08-0.eryxLJ > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
        except:
            continue


    lista_conocidos = []
    try:
        for i in range(1,5):
            nuevo_conocido = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').get_attribute('href').split('/')[5]
            lista_conocidos.append(nuevo_conocido)                                         
    except:
        pass

    actores.update_one({'nombre':actor['nombre']},{'$set': {'conocido': lista_conocidos}})



        


driver.close()


100%|██████████| 5938/5938 [26:21<00:00,  3.75it/s]


In [28]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/")

for pelicula in tqdm(list(peliculas.find())):
    for actor in pelicula['actores']:
        if actores.find_one({'nombre': actor}):
            continue
        dicc = {}
        dicc['nombre'] = actor
        #actores.update_one({'nombre': actor},{'$addToSet': {'peliculas': pelicula['id_IMDB']}})
        try:
            busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor, Keys.ENTER)
            driver.implicitly_wait(3)
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-da7f4a08-0.eryxLJ > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
    
        except:
            driver.close()
            #Abrimos el Chrome
            driver = webdriver.Chrome()

            # Maximizar la ventana del navegador
            driver.maximize_window()

            # Navegar a la página web de IMDB 
            driver.get("https://www.imdb.com")
            busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor, Keys.ENTER)
            try:
                driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-da7f4a08-0.eryxLJ > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
            except:
                continue

        try: 
            dicc['year'] = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-c0933c3e-1.fQDAuW.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(22) > div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(3) > div > ul > li.ipc-inline-list__item.test-class-react > a").text.split(",")[-1] 
            dicc['year'] = int(dicc['year'].split()[-1])
                                                    #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-11.kkLqLt > section > aside > div > span:nth-child(2)
        except:
            dicc['year'] = np.nan
        try:
            lista_conocidos = []
            for i in range(1,5):
                conocido = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').get_attribute('href').split('/')[5]
                lista_conocidos.append(conocido)
            dicc['conocido'] =lista_conocidos                   #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid
        except:
            dicc['conocido']= lista_conocidos
        #Esto hay que cambiarlo para que saque todas las cosas que pone, pero como un único str
        try:
            dicc['roles'] = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div > ul").text.split("\n")
        except:
            dicc['roles']= np.nan
        #Faltan premios (solo si tiene oscar)
        try:
            dicc['premios'] = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-c0933c3e-1.fQDAuW.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > div > ul > li > span').text

            try:
                dicc['premios'] = int(dicc['premios'].split(' premio')[0])
                nominaci_y =dicc['premios'].split(' premio')[-1].split(' nominaci')[0]
                dicc['nominaciones'] = int(nominaci_y.split(' y ')[-1])
            
            except:
                if 'nominaci' in dicc['premios']: 
                    dicc['nominaciones'] = int(dicc['premios'].split(' nominaci')[0])
                    dicc['premios'] = np.nan
                else:
                    dicc['premios'] = dicc['premios'].split(' premio')[0]
                    dicc['nominaciones'] = np.nan





        except:
            dicc['premios'] = np.nan
    
        
        actores.insert_one(dicc)
driver.close()

 12%|█▏        | 3906/31487 [12:38:50<89:18:16, 11.66s/it] 


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#suggestion-search"}
  (Session info: chrome=120.0.6099.234); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001009a84dc chromedriver + 4162780
1   chromedriver                        0x00000001009a0664 chromedriver + 4130404
2   chromedriver                        0x00000001005f7bc0 chromedriver + 293824
3   chromedriver                        0x000000010063e040 chromedriver + 581696
4   chromedriver                        0x000000010067dab0 chromedriver + 842416
5   chromedriver                        0x00000001006316bc chromedriver + 530108
6   chromedriver                        0x0000000100632930 chromedriver + 534832
7   chromedriver                        0x000000010096de08 chromedriver + 3923464
8   chromedriver 